In [3]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/blue-castle/blue_castle.txt


In [4]:
import torch.nn as nn
import numpy as np
import torch

In [5]:
file = open("/kaggle/input/blue-castle/blue_castle.txt", "r", encoding="utf8")
lines = []
for i in file:
    lines.append(i)
data = ""
for i in lines:
    data = " ".join(lines)
data = data.replace('\n', '').replace('\r', '').replace('\ufeff', '').replace('“','').replace('”','')

data = data.split()
data = " ".join(data)
data[:500]

'The Project Gutenberg eBook of The Blue Castle, by Lucy Maud Montgomery This eBook is for the use of anyone anywhere in the United States and most other parts of the world at no cost and with almost no restrictions whatsoever. You may copy it, give it away or re-use it under the terms of the Project Gutenberg License included with this eBook or online at www.gutenberg.org. If you are not located in the United States, you will have to check the laws of the country where you are located before usi'

In [6]:
len(data)

402906

In [7]:
from torchtext.data import get_tokenizer
from torchtext.vocab import build_vocab_from_iterator
tokenizer = get_tokenizer("basic_english")
def preprocess_text(text):
    sequence_data = []
    # tokenize text
    tokens = tokenizer(text)
    def yield_tokens():
        yield tokens
    token_generator = yield_tokens()
    # build vocabulary
    vocab = build_vocab_from_iterator(token_generator)
    # convert it into a sequence based on the tokens
    sequence_data = [vocab[token] for token in tokens]
    print("Length of the sequence_data is ", len(sequence_data))
    return(sequence_data, vocab.get_stoi())
    
    


In [8]:
sequence_data, vocab = preprocess_text(data)
vocab_size = len(vocab)

Length of the sequence_data is  81428


In [9]:
print(vocab_size)

8726


In [10]:
sequences = []
for i in range(3, len(sequence_data)):
    words = sequence_data[i-3:i+1]
    sequences.append(words)
    
print(len(sequences))
sequences = np.array(sequences)
sequences[:10]
    

81425


array([[   2,  112,  244,  640],
       [ 112,  244,  640,    5],
       [ 244,  640,    5,    2],
       [ 640,    5,    2,  104],
       [   5,    2,  104,  148],
       [   2,  104,  148,    1],
       [ 104,  148,    1,   61],
       [ 148,    1,   61, 2996],
       [   1,   61, 2996, 3017],
       [  61, 2996, 3017, 2135]])

In [11]:
X = []
y = []

for i in sequences:
    X.append(i[0:3])
    y.append(i[3])

X = np.array(X)
y = np.array(y)

In [12]:
y = nn.functional.one_hot(torch.tensor(y) , num_classes=vocab_size)
y[:5]

tensor([[0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 1,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0]])

In [26]:
class LSTM(nn.Module):
    def __init__(self, vocab_size, embedding_dim, hidden_dim):
        super(LSTM, self).__init__()
        self.embedding = nn.Embedding(vocab_size, embedding_dim)
        self.lstm1 = nn.LSTM(embedding_dim, hidden_dim, batch_first=True)
        self.lstm2 = nn.LSTM(hidden_dim, hidden_dim, batch_first=True)
        self.dense1 = nn.Linear(hidden_dim, 1000)
        self.relu = nn.ReLU()
        self.dense2 = nn.Linear(1000, vocab_size)
        self.softmax = nn.Softmax()
    
    def forward(self, x):
        x = self.embedding(x)
        x, _ = self.lstm1(x)
        x, _ = self.lstm2(x)
        print(f"Shape after LSTM layers: {x.shape}")
        x = self.dense1(x[:,-1,:])
        x = self.relu(x)
        x = self.dense2(x)
        x = self.softmax(x)
        return x

In [27]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [28]:
device

device(type='cuda')

In [29]:
model = LSTM(vocab_size, 1000, 2000).to(device)

In [30]:
X = torch.tensor(X)
y = torch.tensor(y)

/tmp/ipykernel_34/2519130387.py:1: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  X = torch.tensor(X)
/tmp/ipykernel_34/2519130387.py:2: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  y = torch.tensor(y)


In [31]:
from torch.utils.data import DataLoader, TensorDataset
dataset = TensorDataset(X, y)
batch_size = 10000 
dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=True)

In [114]:
from torch.optim import Adam
from torch.nn import BCELoss

optimizer = Adam(model.parameters(), lr=0.01)
criterion = BCELoss()

epochs = 20
for i in range(epochs):
    model.train()
    for batch_X, batch_y in dataloader:
        batch_X, batch_y = batch_X.to(device), batch_y.to(device)
        y_pred = model(batch_X) 
        loss = criterion(y_pred, batch_y.float())
        
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
    print("Epoch: ", i, "Loss: ", loss)

Shape after LSTM layers: torch.Size([10000, 3, 2000])
Shape after LSTM layers: torch.Size([10000, 3, 2000])
Shape after LSTM layers: torch.Size([10000, 3, 2000])
Shape after LSTM layers: torch.Size([10000, 3, 2000])
Shape after LSTM layers: torch.Size([10000, 3, 2000])
Shape after LSTM layers: torch.Size([10000, 3, 2000])
Shape after LSTM layers: torch.Size([10000, 3, 2000])
Shape after LSTM layers: torch.Size([10000, 3, 2000])
Shape after LSTM layers: torch.Size([1425, 3, 2000])
Epoch:  0 Loss:  tensor(0.0009, device='cuda:0', grad_fn=<BinaryCrossEntropyBackward0>)
Shape after LSTM layers: torch.Size([10000, 3, 2000])
Shape after LSTM layers: torch.Size([10000, 3, 2000])
Shape after LSTM layers: torch.Size([10000, 3, 2000])
Shape after LSTM layers: torch.Size([10000, 3, 2000])
Shape after LSTM layers: torch.Size([10000, 3, 2000])
Shape after LSTM layers: torch.Size([10000, 3, 2000])
Shape after LSTM layers: torch.Size([10000, 3, 2000])
Shape after LSTM layers: torch.Size([10000, 3, 20

In [116]:
def predict_next_word(model, text):
    tokens = tokenizer(text)
    sequence_data = [vocab[token] if token in vocab else 0 for token in tokens]
    sequence_data = np.array(sequence_data)
    sequence_data = torch.tensor(sequence_data).unsqueeze(0).to(device)
    with torch.no_grad():
        preds = model(sequence_data)
    
    preds = preds.cpu().numpy()
    predicted_index = np.argmax(preds)
    print(predicted_index)
    predicted_word = ""
    
    for key, val in vocab.items():
        if val == predicted_index:
            predicted_word = key
            break
    return (predicted_word)
    
    

In [137]:
predict_next_word(model, "the blue")

Shape after LSTM layers: torch.Size([1, 2, 2000])
148


'castle'